Our Sources of data are as follows:


1st dataset: https://data.world/ https://data.world/noaa/severe-weather-data-hail-2017

2nd dqtaset: https://nauticalcharts.noaa.gov/data/us-maritime-limits-and-boundaries.html

In [15]:
#import the requirements
import numpy as np
import pandas as pd
import datetime
import pyodbc
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()


In [16]:
# Import data sources 
hail2017 = "hail2017.csv"
longlat  = "longlat.csv"

In [17]:
# The correct encoding must be used to read the CSV in pandas
dfgeo = pd.read_csv(longlat, encoding="ISO-8859-1")
dfhail2017 = pd.read_csv(hail2017, encoding="ISO-8859-1")

Data Cleaning 
(cleaning, joining, filtering, aggregating, etc).

In [5]:
#convert to date
dfhail2017['ztime']=pd.to_datetime(dfhail2017.ztime)

In [6]:
# Delete extraneous column
dfhail2017.drop(['azimuth','range','cellid','azimuth'], axis=1, inplace=True)
dfhail2017.head()

,ztime,longitude,latitude,wsrid,sevprob,prob,maxsize
0,1970-01-01 05:36:40,-162.21326,22.81952,PHKI,-999,-999,-999.0
1,1970-01-01 05:36:40,-162.12851,22.79089,PHKI,-999,-999,-999.0
2,1970-01-01 05:36:40,-162.11157,22.78516,PHKI,-999,-999,-999.0
3,1970-01-01 05:36:40,-161.97694,22.78461,PHKI,-999,-999,-999.0
4,1970-01-01 05:36:40,-161.96012,22.77859,PHKI,-999,-999,-999.0


In [7]:
dfhail2017.ztime.unique()

array(['1970-01-01T05:36:40.000000000'], dtype='datetime64[ns]')

In [8]:
dfhail2017['ztime'].value_counts()

1970-01-01 05:36:40    1048573
Name: ztime, dtype: int64

In [9]:
#adding breakout of date for further analysis
dfhail2017['year'] = pd.DatetimeIndex(dfhail2017['ztime']).year
dfhail2017['quarter'] = pd.DatetimeIndex(dfhail2017['ztime']).quarter
dfhail2017['month'] = pd.DatetimeIndex(dfhail2017['ztime']).month
dfhail2017['weekofyear'] = pd.DatetimeIndex(dfhail2017['ztime']).weekofyear


dfhail2017.head()

,ztime,longitude,latitude,wsrid,sevprob,prob,maxsize,year,quarter,month,weekofyear
0,1970-01-01 05:36:40,-162.21326,22.81952,PHKI,-999,-999,-999.0,1970,1,1,1
1,1970-01-01 05:36:40,-162.12851,22.79089,PHKI,-999,-999,-999.0,1970,1,1,1
2,1970-01-01 05:36:40,-162.11157,22.78516,PHKI,-999,-999,-999.0,1970,1,1,1
3,1970-01-01 05:36:40,-161.97694,22.78461,PHKI,-999,-999,-999.0,1970,1,1,1
4,1970-01-01 05:36:40,-161.96012,22.77859,PHKI,-999,-999,-999.0,1970,1,1,1


In [10]:
new_df = pd.merge(dfgeo, dfhail2017, on=['wsrid'])
new_df.head()

,wsrid,city,statename,ztime,longitude,latitude,sevprob,prob,maxsize,year,quarter,month,weekofyear
0,KABR,ABERDEEN,SD,1970-01-01 05:36:40,-101.95171,46.06155,-999,-999,-999.0,1970,1,1,1
1,KABR,ABERDEEN,SD,1970-01-01 05:36:40,-101.90497,46.05434,-999,-999,-999.0,1970,1,1,1
2,KABR,ABERDEEN,SD,1970-01-01 05:36:40,-101.84419,46.08932,-999,-999,-999.0,1970,1,1,1
3,KABR,ABERDEEN,SD,1970-01-01 05:36:40,-101.77436,46.07758,-999,-999,-999.0,1970,1,1,1
4,KABR,ABERDEEN,SD,1970-01-01 05:36:40,-101.66676,46.10190,-999,-999,-999.0,1970,1,1,1


In [12]:
new_df1 = new_df.groupby('statename')['wsrid'].count()
new_df1.head()

statename
 AK      261
 AL    90405
 AR    32829
 AZ     4374
 CA     5650
Name: wsrid, dtype: int64

In [18]:
rds_connection_string = "root:Sandiego082107!!@127.0.0.1/hail_db"
engine = create_engine(f'mysql://{rds_connection_string}')


In [19]:
#loading the data from dataframe to sql
new_df.to_sql(name='geolocation', con=engine, if_exists='append', index=False)

In [15]:
#confirm that your data has been loaded to sql

pd.read_sql_query('select * from geolocation', con=engine).head()

,wsrid,city,statename,ztime,longitude,latitude,sevprob,prob,maxsize,year,quarter,month,weekofyear
0,KABR,ABERDEEN,SD,1970-01-01 05:36:40,-101.95171,46.06155,-999,-999,-999.0,1970,1,1,1
1,KABR,ABERDEEN,SD,1970-01-01 05:36:40,-101.90497,46.05434,-999,-999,-999.0,1970,1,1,1
2,KABR,ABERDEEN,SD,1970-01-01 05:36:40,-101.84419,46.08932,-999,-999,-999.0,1970,1,1,1
3,KABR,ABERDEEN,SD,1970-01-01 05:36:40,-101.77436,46.07758,-999,-999,-999.0,1970,1,1,1
4,KABR,ABERDEEN,SD,1970-01-01 05:36:40,-101.66676,46.10190,-999,-999,-999.0,1970,1,1,1


In [16]:
#confirm data by querying sql
pd.read_sql_query('select * from geolocation', con=engine).head()

,wsrid,city,statename,ztime,longitude,latitude,sevprob,prob,maxsize,year,quarter,month,weekofyear
0,KABR,ABERDEEN,SD,1970-01-01 05:36:40,-101.95171,46.06155,-999,-999,-999.0,1970,1,1,1
1,KABR,ABERDEEN,SD,1970-01-01 05:36:40,-101.90497,46.05434,-999,-999,-999.0,1970,1,1,1
2,KABR,ABERDEEN,SD,1970-01-01 05:36:40,-101.84419,46.08932,-999,-999,-999.0,1970,1,1,1
3,KABR,ABERDEEN,SD,1970-01-01 05:36:40,-101.77436,46.07758,-999,-999,-999.0,1970,1,1,1
4,KABR,ABERDEEN,SD,1970-01-01 05:36:40,-101.66676,46.10190,-999,-999,-999.0,1970,1,1,1
